# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from pprint import pprint


# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/sample_cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

615

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# Sample of Json response from API Call
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
sample_cities = ["irvine", "bangkok", "tokyo", "taipei", "manila", "hong kong"]

params = {"appid": api_key,
         "units": units,
         "q": "irvine"}

response = requests.get(url, params=params).json()
pprint(response)

{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 33.69, 'lon': -117.83},
 'dt': 1536375480,
 'id': 5359777,
 'main': {'humidity': 73,
          'pressure': 1013,
          'temp': 73.27,
          'temp_max': 80.6,
          'temp_min': 66.2},
 'name': 'Irvine',
 'sys': {'country': 'US',
         'id': 485,
         'message': 0.0058,
         'sunrise': 1536413408,
         'sunset': 1536458823,
         'type': 1},
 'visibility': 16093,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 200, 'speed': 5.82}}


In [4]:
#Create Dataframe
weather_df = pd.DataFrame({"City": sample_cities})
weather_df["Country"] = ""
weather_df["Temperature (F)"] = ""
weather_df["Latitude"] = ""
weather_df["Longitude"] = ""
weather_df["Humidity (%)"] = ""
weather_df["Cloudiness (%)"] = ""
weather_df["Wind Speed (mph)"] = ""
weather_df

,City,Country,Temperature (F),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
0,irvine,,,,,,,
1,bangkok,,,,,,,
2,tokyo,,,,,,,
3,taipei,,,,,,,
4,manila,,,,,,,
5,hong kong,,,,,,,


In [5]:
# Perform API Calls for each City
params = {"appid": api_key,
         "units": units,
         "q": ""}

weather = []

print("Beginning Data Retrieval")
print("----------------------------------")

for index, row in weather_df.iterrows():
    params["q"] = row.City
    print("Processing Record " + str(index + 1) + " of " + str(len(sample_cities)) + " | " + row.City)
    response = requests.get(url, params=params).json()
    weather.append(response)
    
    try:
        weather_df.loc[index, "Country"] = response['sys']['country']
        weather_df.loc[index, "Temperature (F)"] = response['main']['temp']
        weather_df.loc[index, "Latitude"] = response['coord']['lat']
        weather_df.loc[index, "Longitude"] = response['coord']['lon']
        weather_df.loc[index, "Humidity (%)"] = response['main']['humidity']
        weather_df.loc[index, "Cloudiness (%)"] = response['clouds']['all']
        weather_df.loc[index, "Wind Speed (mph)"] = response['wind']['speed']
    except:
        print("City Not Found...Sipping...")
        
    
          
print("----------------------------------")
print("Data Retrieval Complete")
print("----------------------------------")

Beginning Data Retrieval
----------------------------------
Processing Record 1 of 6 | irvine
Processing Record 2 of 6 | bangkok
Processing Record 3 of 6 | tokyo
Processing Record 4 of 6 | taipei
Processing Record 5 of 6 | manila
Processing Record 6 of 6 | hong kong
----------------------------------
Data Retrieval Complete
----------------------------------


In [6]:
weather_df

,City,Country,Temperature (F),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
0,irvine,US,73.27,33.69,-117.83,73,1,5.82
1,bangkok,TH,84.2,13.75,100.49,74,40,9.17
2,tokyo,JP,88.99,35.68,139.76,63,75,23.04
3,taipei,TW,84.87,25.04,121.56,70,75,6.93
4,manila,PH,86,14.59,120.98,84,40,9.17
5,hong kong,CN,84.7,22.28,114.16,62,20,12.75


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot